In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import models
from torchvision.models.vgg import VGG
from PIL import Image
import numpy as np
import os
import nibabel as nib
import time
import datetime
import torch.nn.functional as F
from models.unet import UNet
import matplotlib.pyplot as plt
from metrics.torch_seg_metrics import dice_score, iou
from datasets.BRATS2018 import ToTensorVal, NormalizeBRATSVal, ZeroPadVal
from tqdm import tqdm

In [2]:
# helper functions
def time_stamp() -> str:
    ts = time.time()
    time_stamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    return time_stamp

In [3]:
def infer(case_name, seg_type, model_path, device):
    case_dir = os.path.join(val_dir, case_name)
    if seg_type == 'et' or seg_type == 'tc':
        scan_path = os.path.join(case_dir, case_name + '_t1ce.nii.gz')
        sc = nib.load(scan_path)
        sc = sc.get_data()
        sc = sc.transpose((2, 0, 1))
        assert sc.shape == (155, 240, 240)
        
        model = UNet(n_channels=1, n_classes=1)
    elif seg_type == 'wt':
        t2_path = os.path.join(case_dir, case_name + '_t2.nii.gz')
        flair_path = os.path.join(case_dir, case_name + '_flair.nii.gz')
        t2 = nib.load(t2_path).get_data()
        flair = nib.load(flair_path).get_data()
        sc = np.array([t2, flair]).transpose((3, 0, 1, 2))
        assert sc.shape == (155, 2, 240, 240)
        
        model = UNet(n_channels=2, n_classes=1)
    else:
        raise ValueError('seg_type should only be et, tc or wt')
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    
    preds = np.zeros((155, 240, 240))
    
    for i in range(155):
        slice_i = sc[i]
        if not seg_type == 'wt':
            slice_i = np.expand_dims(slice_i, axis=0)
            assert slice_i.shape == (1, 240, 240)
        else:
            assert slice_i.shape == (2, 240, 240)
        
        normalize = NormalizeBRATSVal()
        totensor = ToTensorVal()
        
        slice_i = normalize(slice_i)
        slice_i = totensor(slice_i)
        
        # unsqueeze the dimension to 4, NxCxHxW
        slice_i = torch.unsqueeze(slice_i, dim=0)
        slice_i = slice_i.to(device)
        with torch.no_grad():
            output = model(slice_i)
            pred = torch.sigmoid(output) > 0.5
        
        # squeeze the dimension down to 2, HxW
        pred = torch.squeeze(pred)
        pred = pred.cpu().numpy()
        preds[i] = pred
            
    
    np.save(os.path.join(case_dir, case_name + f'_{seg_type}'), preds)
    return preds
    

# sorted validation cases

In [53]:
case_list = sorted(os.listdir('BRATS2018_Validation/'))
val_dir = 'BRATS2018_Validation/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if not os.path.exists('validations/'):
    os.makedirs('validations/')

In [9]:
infer('Brats18_CBICA_AAM_1',\
      'wt',\
      '../UNet_BraTS_DiceLoss_WT_210_manual/UNets-BRATS2018-WT-210-T2-Flair_batch6_training_epochs10_Adam_scheduler-step10-gamma1.0_lr1e-05_w_decay1e-05/trained_model.pt',\
      device)

torch.Size([240, 240]) 0
torch.Size([240, 240]) 1
torch.Size([240, 240]) 2
torch.Size([240, 240]) 3
torch.Size([240, 240]) 4
torch.Size([240, 240]) 5
torch.Size([240, 240]) 6
torch.Size([240, 240]) 7
torch.Size([240, 240]) 8
torch.Size([240, 240]) 9
torch.Size([240, 240]) 10
torch.Size([240, 240]) 11
torch.Size([240, 240]) 12
torch.Size([240, 240]) 13
torch.Size([240, 240]) 14
torch.Size([240, 240]) 15
torch.Size([240, 240]) 16
torch.Size([240, 240]) 17
torch.Size([240, 240]) 18
torch.Size([240, 240]) 19
torch.Size([240, 240]) 20
torch.Size([240, 240]) 21
torch.Size([240, 240]) 22
torch.Size([240, 240]) 23
torch.Size([240, 240]) 24
torch.Size([240, 240]) 25
torch.Size([240, 240]) 26
torch.Size([240, 240]) 27
torch.Size([240, 240]) 28
torch.Size([240, 240]) 29
torch.Size([240, 240]) 30
torch.Size([240, 240]) 31
torch.Size([240, 240]) 32
torch.Size([240, 240]) 33
torch.Size([240, 240]) 34
torch.Size([240, 240]) 35
torch.Size([240, 240]) 36
torch.Size([240, 240]) 37
torch.Size([240, 240])

In [54]:
for case_name in tqdm(case_list):
    preds_wt = infer(case_name, 'wt', '../UNet_BraTS_DiceLoss_WT_210_manual/UNets-BRATS2018-WT-210-T2-Flair_batch6_training_epochs10_Adam_scheduler-step10-gamma1.0_lr1e-05_w_decay1e-05/trained_model.pt',\
         device)
    assert preds_wt.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name} whole tumor.')
    
    preds_et = infer(case_name, 'et', '../UNet_BraTS_DiceLoss_ET_manual/UNets-BRATS2018-ET-DiceLoss_batch6_training_epochs50_Adam_scheduler-step10-gamma1.0_lr0.0001_w_decay1e-05/terminated_model.pt',\
         device)
    assert preds_et.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name} enhancing tumor.')
    
    preds_tc = infer(case_name, 'tc', '../UNet_BraTS_DiceLoss_TC_manual/UNets-BRATS2018-TC-DiceLoss_batch6_training_epochs50_Adam_scheduler-step10-gamma1.0_lr1e-05_w_decay1e-05/terminated_model.pt',\
         device)
    assert preds_tc.shape == (155, 240, 240)
    print(f'{time_stamp()} Finish {case_name} tumor core.')
    
    preds_2 = np.logical_xor(preds_wt, preds_tc).astype(np.uint8)
    preds_1 = np.logical_xor(preds_tc, preds_et).astype(np.uint8)
    preds_4 = preds_et.astype(np.uint8)
    
    preds_seg = 4 * preds_4 + 2 * preds_2 + preds_1
    assert preds_seg.shape == (155, 240, 240)
    
    # get the mask of which computed label is greater than 4, and set it to 4
    preds_seg = preds_seg + (preds_seg > 4).astype(np.uint8) * (4 * np.ones((155, 240, 240)) - preds_seg)
    preds_seg = preds_seg + (preds_seg == 3) * np.ones((155, 240, 240))
    preds_seg = preds_seg.astype(np.uint8)
    print(np.unique(preds_seg))
    
    img = nib.Nifti1Image(preds_seg.transpose(1, 2, 0), affine=None)
    nib.save(img, os.path.join('validations', case_name + '_seg.nii.gz'))
    
    



  0%|          | 0/66 [00:00<?, ?it/s]

2019-05-01 15:25:21 Finish Brats18_CBICA_AAM_1 whole tumor.
2019-05-01 15:25:37 Finish Brats18_CBICA_AAM_1 enhancing tumor.
2019-05-01 15:25:53 Finish Brats18_CBICA_AAM_1 tumor core.




  2%|▏         | 1/66 [00:49<53:23, 49.29s/it]

[0 1 2 4]
2019-05-01 15:26:11 Finish Brats18_CBICA_ABT_1 whole tumor.
2019-05-01 15:26:27 Finish Brats18_CBICA_ABT_1 enhancing tumor.
2019-05-01 15:26:43 Finish Brats18_CBICA_ABT_1 tumor core.




  3%|▎         | 2/66 [01:39<52:44, 49.45s/it]

...

[0 1 2 4]
2019-05-01 16:19:45 Finish Brats18_WashU_W047_1 whole tumor.
2019-05-01 16:20:01 Finish Brats18_WashU_W047_1 enhancing tumor.
2019-05-01 16:20:18 Finish Brats18_WashU_W047_1 tumor core.




 98%|█████████▊| 65/66 [55:14<00:51, 51.87s/it]

[0 1 2 4]
2019-05-01 16:20:36 Finish Brats18_WashU_W053_1 whole tumor.
2019-05-01 16:20:53 Finish Brats18_WashU_W053_1 enhancing tumor.
2019-05-01 16:21:09 Finish Brats18_WashU_W053_1 tumor core.




100%|██████████| 66/66 [56:05<00:00, 51.62s/it]

[0 1 2 4]
